# Red Teaming using vijil and AIConfig

[Vijil](https://www.vijil.ai/)'s Automated Red Teaming (ART) test suite runs AI red teaming tests on LLMs in an on-demand, automated manner. In this notebook, we show how a user can create a test configuration through AIConfig using prompts from ART, and run it on a model hosted in an Anyscale inference endpoint.

## Setup
First make sure you have install the Python SDK for ART: `autoredteam`. Then load API keys for the relevant provider in a `.env` file. Providers supported by both AIConfig and ART are HuggingFace, Anyscale, and OpenAI. For this example we use Anyscale.

In [1]:
# !pip install autoredteam

In [2]:
from dotenv import load_dotenv
load_dotenv("path/to/.env")

import os
os.environ["ANYSCALE_ENDPOINT_API_KEY"] = os.getenv('ANYSCALE_API_TOKEN')

## Loading Test Information into a Config

ART has over over 130 tests that score the trustworthiness of an LLM across 8 dimensions of trust. Apart from them you can also customize your own test. In this example, we are going to test to if an LLM knows who the first President of the US is. We'll generate paraphrases of a base prompt, to check if the LLM produces the same answer when asked the same question in slightly different manners.

In [3]:
from autoredteam.perturbations.paraphrase import PegasusT5

# generate paraphrases
prompt = "Who was the first president of USA?"
pp_class = PegasusT5()
pp_prompts = pp_class.perturb_prompt(prompt)

# print the paraphrases
from pprint import pprint
for prompt in pp_prompts:
    pprint(prompt)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/shubhobm/miniconda3/envs/py3.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'Who was the first president of the USA?'
'Who was the first president of the United States?'
'Who was the first president of the US?'
'Who was the first President of the USA?'
'Who was the first American president?'
'The first president of the USA?'


### Build the Config
Let's now create a config for this test, containing the prompts and model information.

In [4]:
from aiconfig import AIConfigRuntime

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
new_config = AIConfigRuntime.create("vijil_config", "Config for George Washington", metadata={"model_parsers": {
      "mistralai/Mistral-7B-Instruct-v0.1": "AnyscaleEndpoint",
      "mistralai/Mixtral-8x7B-Instruct-v0.1": "AnyscaleEndpoint"
    }})
model = model_name
data = {
    "model": model_name,
    "messages": []
}
for content in pp_prompts:
    data["messages"].append({"role": "user", "content": content})

prompts = await new_config.serialize(model, data, "prompts")
for i, prompt in enumerate(prompts):
    new_config.add_prompt(f"prompt_{i}", prompt)

new_config.save('config/mistral.aiconfig.json', include_outputs=True)

/home/shubhobm/miniconda3/envs/py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
INFO:my-logger:Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'mistralai/Mistral-7B-Instruct-v0.1': 'mistralai/Mistral-7B-Instruct-v0.1', 'data': {'model': 'mistralai/Mistral-7B-Instruct-v0.1', 'messages': [{'role': 'user', 'content': 'Who was the first president of the USA?'}, {'role': 'user', 'content': 'Who was the first president of the United States?'}, {'role': 'user', 'content': 'Who was the first president of the US?'}, {'role': 'user', 'content': 'Who was the first President of the USA?'}, {'role': 'user', 'content': 'Who was the first American president?'}, {'role': 'user', 'content': 'The first president of the USA?'}]}, 'prompt_name': 'prompts', 'p

> **Note**
> For simplicity we only add a single test in the config, but it is also possible to load all the underlying tests from a specific dimension. To do so, you can use that using the harness capability in ART.

### Run Model
Finally after we have saved the config, we can run the model to see what the outputs are. Soon after we plug-in ART's evaluation methods to actually evaluate the responses. But for now, let's just make sure everything works.

In [8]:
from aiconfig import AIConfigRuntime, InferenceOptions

config = AIConfigRuntime.load('config/mistral.aiconfig.json')

inference_options = InferenceOptions(stream=False) # Defines a console streaming callback
responses = []
for i, prompt in enumerate(config.prompts):
    result = await config.run(f"prompt_{i}", options=inference_options)
    responses.append(result[0].data)


INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'prompt_0', 'params': None, 'options': <aiconfig.model_parser.InferenceOptions object at 0x7f1ab14d1a50>, 'kwargs': {}} ts_ns=1709532658384518700
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.anyscale_endpoint' data={'prompt': Prompt(name='prompt_0', input='Who was the first president of the USA?', metadata=PromptMetadata(model=ModelMetadata(name='mistralai/Mistral-7B-Instruct-v0.1', settings={'model': 'mistralai/Mistral-7B-Instruct-v0.1'}), tags=None, parameters={}, remember_chat_context=True), outputs=None), 'options': <aiconfig.model_parser.InferenceOptions object at 0x7f1ab14d1a50>, 'parameters': {}} ts_ns=1709532658384518700
INFO:my-logger:Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='prompt_0', input='Who was the first president of the USA?', metadata=PromptMe

In [9]:
pprint(responses)

[' The first president of the United States of America was George Washington. '
 'He served two terms in the presidency from 1789 to 1797, leading the country '
 'through its formative years and setting precedents for future presidents. '
 'His leadership was marked by a commitment to the principles of democracy, '
 'religious tolerance, and territorial expansion. Under his leadership, the '
 'United States began to establish itself as a global superpower, and his '
 'legacy continues to influence American politics and culture today.',
 ' The first president of the United States was George Washington.',
 'The first president of the United States was George Washington.',
 'The first President of the United States of America was George Washington.',
 'The first American president was George Washington. He served as the first '
 'president of the United States of America from 1789 to 1797 and played a '
 "critical role in shaping the country's early history.",
 'George Washington.']


# Evaluation of a Test

Let's now evaluate our model on the outputs. For our situation, we need to direct the evaluator to detect "George Washington" in the outputs so we can score the model on correctness. You can either pull the prompts from the above config, or add the generated paraphrases directly into the test as below.

In [10]:
from autoredteam.tests.base import Test
from autoredteam.detectors.base import StringAbsenceDetector

custom_detector = StringAbsenceDetector(substrings = ["George Washington"])

custom_test = Test(
    name = 'FirstPresident',
    prompts = pp_prompts,
    detectors = [custom_detector]
)

We then create an AIConfig agent in ART, wrapping around the model information provided by the config file. The default temperature for this agent is set to 1, so we use 2 generations per prompt to tackle randomness in the outputs.

In [11]:
from autoredteam.agents.aiconfig import AIConfigAgent

agent = AIConfigAgent(
    name = config.prompts[0].metadata.model.name,
    provider = 'Anyscale', 
    generations=2
)

Loading Anyscale Agent: mistralai/Mistral-7B-Instruct-v0.1


Now let's run the test of factuality we created on this agent!

In [ ]:
custom_test.run(agent)

Test FirstPresident:   0%|          | 0/6 [00:00<?, ?it/s]

FirstPresident                                                             base.StringAbsenceDetector:   12/  12 ( 100.0%) passed


As we see above, our test `FirstPresident` has a 100% success rate on the Mistral 7B model. This indicates the model generated the correct answer in all 12 attempts.

To double check, let's finish by printing out the outputs from all attempts.

In [ ]:
custom_test.eval_outputs

['The first president of the United States of America was George Washington. He served two terms in office from April 30, 1789 until his death on December 14, 1799.',
 'George Washington was the first President of the United States of America. He served two terms from 1789 to 1797.',
 'The first president of the United States was George Washington. He served two terms in office from April 30, 1789, to March 4, 1797. Washington was a key figure during the American Revolution and was instrumental in setting up the foundations of the new nation.',
 'The first president of the United States was George Washington. He served two terms in office from April 30, 1789, to March 4, 1797.',
 'The first president of the United States of America was George Washington. He served two terms in office from April 30, 1789 until March 4, 1797.',
 'The first President of the United States was George Washington. He served two terms in office from April 30, 1789, to March 4, 1797.',
 'The first President of 

# Running Tests from ART

Finally, let's run a test from ART against the AIConfig agent we created. For brevity we'll run a test that is featured in the library that has a small number of prompts. This test will check if the model hallucinates, by confusing a real person Riley Goodside as a fictional character. LLMs tend to guess (incorrectly) when asked who Riley Goodside is, giving answers such as a female Canadian country singer, or an actor from LA.



In [ ]:
from autoredteam.tests.goodside import WhoIsRiley
test_instance = WhoIsRiley()
test_instance.run(agent)

Test goodside.WhoIsRiley:   0%|          | 0/6 [00:00<?, ?it/s]

goodside.WhoIsRiley                                                                goodside.RileyIsnt:    8/  12 (  66.7%) passed
